# I] Import and download

In [2]:
import pandas, numpy
import scipy, scipy.signal
import matplotlib, matplotlib.pyplot as plt

In [3]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

In [4]:
path = output_file_directory + "dataframes_for_input/"

In [ ]:
%%time
# DL data gene expression for Normal Tissue and Primary Tumors
df_normal = pandas.read_csv(path+"ensembl_normal_protcoding_expr.tsv", sep = "\t")
df_primary = pandas.read_csv(path+"ensembl_primary_protcoding_expr.tsv", sep = "\t")

print(df_normal.shape)
print(df_primary.shape)

df_normal.head(2)

# II] Data transformation

In [ ]:
#remove gene where the maximum is lower than 10 TPM 
dfs =  df_normal, df_primary
infos = 'normal', 'primary'

df_L = []
for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    
    print(f'\n {info.upper()} \nshape is: {df.shape}')
    #remove gene where the maximum is lower than 10 TPM 
    max_number_log2_tpm_p1 = 3.322
    df = df.loc[:, (df.max() > max_number_log2_tpm_p1)]
    print(f'minimum of the max expression is {df.max().min()} [log2(tpm+1)]')
    print(f'new shape is: {df.shape}')
    df_L.append(df)

df_normal = df_L[0].T
df_primary = df_L[1].T

In [ ]:
df_normal.head()

# III] Distribution and filtering

In [ ]:
# find max
print(df_normal.max().max())
print(df_primary.max().max())

In [ ]:
%%time
#show everything. All the samples 
found_max = 19
#means 50 dots per bins
resolution = 50
number_of_bins = found_max * resolution
absolute_max = 0
#remove at extremities
margin = int(resolution/3)

dfs = df_normal, df_primary
infos = "normal", "primary"

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]  

    for sample in df.columns:
        expression_values = df.loc[:, sample]

        # histogram
        hist, bin_edges = numpy.histogram(expression_values, bins=number_of_bins, range=(0, found_max))
        half_bin = (bin_edges[1] - bin_edges[0])/2
        x = bin_edges + half_bin
        x = x[:-1]

        # curve fitting - smooth
        plotting_x = x[margin:-margin]
        plotting_hist = hist[margin:-margin]
        yhat = scipy.signal.savgol_filter(plotting_hist, 51, 3)
        
        # plotting
        plt.plot(plotting_x, yhat, '-', lw=4, alpha=1/300, color='tab:blue')

    plt.xlim(-1, 11)                  
    plt.xticks(range(0, 10+1))
    plt.title(f'Distribution of of the expression in {info} for all samples ')
    plt.xlabel('Expression [log$_2$ (TPM + 1)]')
    plt.ylabel('Feature count')
    plt.grid(ls=':')
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
%%time
#Plot the maximum of expression to find a communal pattern among samples
#because we saw bad sample in the beginning until 1. I remove the 50 values first with margin

found_max = 19
# means 50 dots per bins (betweeen 0 and 1)
resolution = 50
number_of_bins = found_max * resolution
absolute_max = 0

dfs = df_normal, df_primary
infos = "normal", "primary"

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]  
    
    most_likely_expressions = []
    for sample in df.columns:
        expression_values = df.loc[:, sample]

        # histogram of all sample (without the beginning of the values = no pic at 0)
        hist, bin_edges = numpy.histogram(expression_values, bins=number_of_bins, range=(0, found_max))
        half_bin = (bin_edges[1] - bin_edges[0])/2
        x = bin_edges + half_bin
        x = x[:-1]
        
        # curve fitting
        margin = 50
        plotting_x = x[margin:-margin]
        plotting_hist = hist[margin:-margin]
        yhat = scipy.signal.savgol_filter(plotting_hist, 51, 3)
        
        # To visualize the communal pattern in samples : maximum of all samples 
        ## argmax gives position of the max.
        most_likely_expression = x[numpy.argmax(yhat)]
        most_likely_expressions.append(most_likely_expression)
        
    # histogram of the maximum value of all samples
    print(len(most_likely_expressions))
    hist, bin_edges = numpy.histogram(most_likely_expressions, bins=250, range=(0, 10))
    half_bin = (bin_edges[1] - bin_edges[0])/2
    x = bin_edges + half_bin
    x = x[:-1]

    # curve fitting
    margin = 5
    plotting_x = x[margin:-margin]
    plotting_hist = hist[margin:-margin]
    yhat = scipy.signal.savgol_filter(plotting_hist, 51, 3)

    #plot
    plt.plot(x, hist, 'o', color='tab:blue', alpha=1/2, markeredgecolor='None', markersize=10)
    plt.plot(plotting_x, yhat, '-', lw=4, alpha=1/2, color='tab:blue')

    plt.xticks(range(0, 11))
    plt.grid(ls=':')
    plt.title(f'Distribution of the maximum in {info} for all samples ')
    plt.xlabel('Most expected log2 TPM + 1')
    plt.ylabel('Sample count')
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
print('The communal pattern of the maximal is mainly between 1.8 and 3.8 in the two groups')

In [ ]:
%%time
# We want to select only the samples following the pattern.

found_max = 19
#means 50 dots per bins
resolution = 50
number_of_bins = found_max * resolution
absolute_max = 0
margin = 50

dfs = df_normal, df_primary
infos = "normal", "primary"

selected_expressions = []

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    
    print(info)
    selected_samples = []
    
    for sample in df.columns:
        expression_values = df.loc[:, sample]

        # histogram
        hist, bin_edges = numpy.histogram(expression_values, bins=number_of_bins, range=(0, found_max))
        half_bin = (bin_edges[1] - bin_edges[0])/2
        x = bin_edges + half_bin
        x = x[:-1]

        # curve fitting - smooth
        plotting_x = x[margin:-margin]
        plotting_hist = hist[margin:-margin]
        yhat = scipy.signal.savgol_filter(plotting_hist, 51, 3)
        
        # determine most likely expression
        ## argmax gives position of the max. This is to select samples with max at this range
        most_likely_expression = x[numpy.argmax(yhat)]

        # sample selection. Where max is between 1.8 and 3.8.
        if 1.8 <= most_likely_expression <= 3.8:
            selected_samples.append(sample)
            
    selected_expression = df.loc[:, selected_samples]
    print(selected_expression.shape)
    selected_expressions.append(selected_expression)

# create the new dataframe
selected_expr_normal = selected_expressions[0]
selected_expr_primary = selected_expressions[1]
selected_expr_normal.head(3)

In [ ]:
%%time
#Plot the distribution of the selected samples
found_max = 19
#means 50 dots per bins
resolution = 50
number_of_bins = found_max * resolution
absolute_max = 0
#remove at extremities
margin = int(resolution/3)

dfs = selected_expr_normal, selected_expr_primary
infos = "normal", "primary"

most_likely_expressions = []
selected_samples = []

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]  

    for sample in df.columns:
        expression_values = df.loc[:, sample]

        # histogram
        hist, bin_edges = numpy.histogram(expression_values, bins=number_of_bins, range=(0, found_max))
        half_bin = (bin_edges[1] - bin_edges[0])/2
        x = bin_edges + half_bin
        x = x[:-1]

        # curve fitting - smooth
        plotting_x = x[margin:-margin]
        plotting_hist = hist[margin:-margin]
        yhat = scipy.signal.savgol_filter(plotting_hist, 51, 3)
               
        # plotting
        plt.plot(plotting_x, yhat, '-', lw=4, alpha=1/300, color='tab:green')

    plt.xlim(-1, 11)                  
    plt.xticks(range(0, 10+1))
    plt.title(f'Distribution of the expression in {info}')
    plt.xlabel('Expression [log$_2$ (TPM + 1)]')
    plt.ylabel('Feature count')
    plt.grid(ls=':')
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
selected_expr_normal = selected_expr_normal.T
selected_expr_primary = selected_expr_primary.T

print(selected_expr_normal.shape)
print(selected_expr_primary.shape)

In [ ]:
# %%time
# #save to csv
# selected_expr_normal.to_csv(path+'filtered_samples_protcod_expr_normal.tsv',sep = "\t")
# selected_expr_primary.to_csv(path+'filtered_samples_protcod_expr_protein.tsv',sep = "\t")